# 🧠 CNN con Estimación de Incertidumbre (Epistémica + Aleatoria)

## Pipeline de Entrenamiento e Inferencia con Cuantificación de Incertidumbre

Este notebook implementa un modelo CNN con estimación de dos tipos de incertidumbre:

### 🎯 Tipos de Incertidumbre:
1. **Epistémica (modelo)**: Capturada con MC Dropout → reducible con más datos
2. **Aleatoria (datos)**: Capturada con cabeza de varianza → irreducible

### 📋 Pipeline:
1. **Setup**: Configuración del entorno
2. **Data Loading**: Carga de datos desde cache (augmentados)
3. **Split**: Train/Val/Test estratificado (70/15/15)
4. **Model**: CNN con DOS cabezas (predicción + ruido)
5. **Training**: Pérdida heteroscedástica con ruido gaussiano
6. **Evaluation**: MC Dropout para epistémica + σ² para aleatoria
7. **Visualization**: Histogramas, reliability plot, scatter

### 🏗️ Arquitectura:
- **Backbone**: 2× bloques Conv2D → BN → ReLU → MaxPool(3×3) → Dropout
- **Cabeza A**: Predicción (logits ∈ ℝ^C)
- **Cabeza B**: Ruido de datos (s_logit = log σ² ∈ ℝ^C, clamped [-10, 3])
- **MC Dropout**: Permanece activo en inferencia

### ⚙️ Entrenamiento:
- **Pérdida**: Heteroscedástica (log-likelihood con ruido gaussiano)
- **T_noise**: 5 muestras de ruido en entrenamiento
- **Optimizer**: AdamW (lr=1e-3, wd=1e-4)
- **Early stopping**: Por val_loss

### 🔬 Inferencia:
- **T_test**: 30-50 pasadas MC Dropout
- **Epistémica**: BALD = H(p̄) - mean H(p_t)
- **Aleatoria**: mean σ²_t
- **Total**: Entropía H(p̄)

### ⚠️ PREREQUISITO:
**Ejecutar primero `data_preprocessing.ipynb`** para generar el cache de datos preprocesados.


## 1. Setup y Configuración

In [1]:
# ============================================================
# IMPORTS Y CONFIGURACIÓN
# ============================================================
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Agregar módulos propios al path
sys.path.insert(0, str(Path.cwd()))

# Importar módulos propios
from modules.augmentation import create_augmented_dataset
from modules.dataset import to_pytorch_tensors
from modules.uncertainty_model import UncertaintyCNN, print_uncertainty_model_summary
from modules.uncertainty_training import train_uncertainty_model, evaluate_with_uncertainty, print_uncertainty_results
from modules.uncertainty_visualization import (
    plot_uncertainty_histograms,
    plot_reliability_diagram,
    plot_uncertainty_scatter,
    plot_training_history_uncertainty
)
from modules.cnn_utils import plot_confusion_matrix

# Configuración de matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configuración de PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Reporte de configuración
print("="*70)
print("🧠 CNN CON ESTIMACIÓN DE INCERTIDUMBRE")
print("="*70)
print(f"✅ Librerías cargadas correctamente")
print(f"🔧 Dispositivo: {device}")
print(f"📦 PyTorch: {torch.__version__}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
print("="*70)

🧠 CNN CON ESTIMACIÓN DE INCERTIDUMBRE
✅ Librerías cargadas correctamente
🔧 Dispositivo: cpu
📦 PyTorch: 2.8.0+cpu


## 2. Carga de Datos

In [2]:
# ============================================================
# CONFIGURACIÓN DE RUTAS
# ============================================================

DATA_PATH_HEALTHY = "./data/vowels_healthy"
DATA_PATH_PARKINSON = "./data/vowels_pk"
CACHE_DIR_HEALTHY = "./cache/healthy"
CACHE_DIR_PARKINSON = "./cache/parkinson"

# Configuración de augmentation
AUGMENTATION_TYPES = ["original", "pitch_shift", "time_stretch", "noise"]
NUM_SPEC_AUGMENT_VERSIONS = 2

print("="*70)
print("📁 CARGANDO DATOS DESDE CACHE")
print("="*70)
print(f"\n⚙️  Configuración:")
print(f"   • Augmentation: {', '.join(AUGMENTATION_TYPES)}")
print(f"   • SpecAugment versions: {NUM_SPEC_AUGMENT_VERSIONS}")

📁 CARGANDO DATOS DESDE CACHE

⚙️  Configuración:
   • Augmentation: original, pitch_shift, time_stretch, noise
   • SpecAugment versions: 2


In [3]:
# ============================================================
# CARGAR DATOS HEALTHY
# ============================================================

print(f"\n🟢 CARGANDO HEALTHY...")
print("="*60)

audio_files_healthy = list(Path(DATA_PATH_HEALTHY).glob("*.egg"))
print(f"   Archivos encontrados: {len(audio_files_healthy)}")

augmented_dataset_healthy = create_augmented_dataset(
    audio_files_healthy,
    augmentation_types=AUGMENTATION_TYPES,
    apply_spec_augment=True,
    num_spec_augment_versions=NUM_SPEC_AUGMENT_VERSIONS,
    use_cache=True,
    cache_dir=CACHE_DIR_HEALTHY,
    force_regenerate=False,
    progress_every=5
)

X_healthy, y_task_healthy, y_domain_healthy, meta_healthy = to_pytorch_tensors(
    augmented_dataset_healthy
)

print(f"\n✅ Healthy cargado:")
print(f"   • Muestras: {X_healthy.shape[0]}")
print(f"   • Shape: {X_healthy.shape}")



🟢 CARGANDO HEALTHY...
   Archivos encontrados: 13
💾 Cargando dataset desde cache...
   📁 ./cache/healthy\augmented_dataset_c6631e32.pkl
✅ Cache cargado exitosamente: 1553 muestras
⚡ Tiempo ahorrado: ~6.5 min
📊 PyTorch tensors listos:
  - X: (1553, 1, 65, 41)
  - y_task: (1553,)  (dist={0: 1553})
  - y_domain: (1553,)  (K dominios=13)

✅ Healthy cargado:
   • Muestras: 1553
   • Shape: torch.Size([1553, 1, 65, 41])


## 3. Modelo con Incertidumbre

In [4]:
# Crear modelo con dos cabezas
model = UncertaintyCNN(
    n_classes=2,
    p_drop_conv=0.25,
    p_drop_fc=0.25,
    input_shape=(65, 41),
    s_min=-10.0,
    s_max=3.0
).to(device)

print_uncertainty_model_summary(model)


RESUMEN DEL MODELO CON INCERTIDUMBRE
UncertaintyCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): MCDropout2d()
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (9): MCDropout2d()
    (10): AdaptiveAvgPool2d(output_size=1)
    (11): Flatten(start_dim=1, end_dim=-1)
  )
  (head_logits): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): MCDropout()
    (3): Linear(in_features=64, out_features=2, bias=True)
  )
  (head_noise): Sequential(
    

## 4. Entrenamiento

In [5]:
# Entrenar con pérdida heteroscedástica
# (continuar con el resto del flujo)